In [169]:
# ! gzip -d '/raid/AISSEL/Hamed/datasets/wit/wit_v1.train.all-00004-of-00010.tsv.gz'
# ! gzip -d '/raid/AISSEL/Hamed/datasets/wit/wit_v1.train.all-00005-of-00010.tsv.gz'
# ! gzip -d '/raid/AISSEL/Hamed/datasets/wit/wit_v1.train.all-00006-of-00010.tsv.gz'
# ! gzip -d '/raid/AISSEL/Hamed/datasets/wit/wit_v1.train.all-00007-of-00010.tsv.gz'
# ! gzip -d '/raid/AISSEL/Hamed/datasets/wit/wit_v1.train.all-00008-of-00010.tsv.gz'
# ! gzip -d '/raid/AISSEL/Hamed/datasets/wit/wit_v1.train.all-00009-of-00010.tsv.gz'

In [1]:
from tqdm.notebook import tqdm, trange

In [2]:
tqdm.pandas()

In [3]:
import sys

In [4]:
import glob
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [5]:
import os
import requests

# General packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import PIL.Image

from IPython.display import Image, display
import warnings
warnings.filterwarnings("ignore")

In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/barzamini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/barzamini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def find_gpus(nums=6):
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp_free_gpus')
    with open('tmp_free_gpus', 'r') as lines_txt:
        frees = lines_txt.readlines()
        idx_freeMemory_pair = [ (idx,int(x.split()[2]))
                              for idx,x in enumerate(frees) ]
    idx_freeMemory_pair.sort(key=lambda my_tuple:my_tuple[1],reverse=True)
    usingGPUs = [str(idx_memory_pair[0])
                    for idx_memory_pair in idx_freeMemory_pair[:nums] ]
    usingGPUs =  ','.join(usingGPUs)
    print('using GPU idx: #', usingGPUs)
    return usingGPUs

In [8]:
os.environ['CUDA_VISIBLE_DEVICES'] = find_gpus(nums=2)

using GPU idx: # 0,1


In [9]:
from random import randint

import numpy as np
# import torch

In [10]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.7394, 0.7871, 0.8458],
        [0.8395, 0.8961, 0.3736],
        [0.8478, 0.3450, 0.9328],
        [0.7653, 0.5004, 0.1363],
        [0.7248, 0.4320, 0.4012]])


In [12]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
os.listdir('/raid/AISSEL/Hamed/datasets/wit/')

['wit_v1.train.all-00007-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00009-of-00010_context_caption_en_sbert_c.tsv',
 '__MACOSX',
 'wit_v1.train.all-1percent_sample.tsv',
 'wit_v1.train.all-00000-of-00010_en_csim.tsv',
 'images',
 'wit_v1.train.all-00000-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00001-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00002-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00003-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00004-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00005-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00006-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00008-of-00010_context_caption_en_sbert_c.tsv']

### Semantic Analysis on caption_reference_description + caption_attribution_description

Also you can consider context_page_description and context_section_description columns for analysis 

### BERTopic 

In [18]:
from bertopic import BERTopic

In [19]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [20]:
lemmatizer = WordNetLemmatizer()

In [21]:
def get_topics(concept='pedestrian'):
    if concept == 'pedestrian':
        model_path = 'ped_auto_model'
    elif concept == 'aircraft':
        model_path = 'aircraft_auto_model'
    elif concept == 'car':
        model_path = 'car_auto_model'
    auto_model = BERTopic(nr_topics="auto")
    auto_model = BERTopic.load(model_path)
    similar_topics, similarity = auto_model.find_topics(concept, top_n=50)

    topics = dict()
    for t in similar_topics:
        l1 = auto_model.get_topic(t)
        el_list = []
        el_list.append(concept)
        for el in l1:
            w = lemmatizer.lemmatize(el[0], get_wordnet_pos(el[0]))
            el_list.append(w)
        topics[t]= set(el_list)
    return topics

# SBERT Semantic Search


In [22]:
from sentence_transformers import SentenceTransformer, util
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def get_sim(u,v):
    emb1 = embedder.encode(u)
#     emb2 = model.encode(v)
    
    return float(util.cos_sim(emb1, v)[0][0])


In [23]:
#Sentences are encoded by calling model.encode()
emb1 = embedder.encode("This is a man with a hat.")
emb2 = embedder.encode("pedestrian")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.2865]])


In [24]:
print("Cosine-Similarity:", get_sim('test 1', embedder.encode('test 2')))

l1 = embedder.encode('pedestrian stripe cross traffic crosswalk signal pedestrian')

Cosine-Similarity: 0.9122401475906372


In [25]:
root_path = '/raid/AISSEL/Hamed/datasets/wit'
os.listdir(root_path)

['wit_v1.train.all-00007-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00009-of-00010_context_caption_en_sbert_c.tsv',
 '__MACOSX',
 'wit_v1.train.all-1percent_sample.tsv',
 'wit_v1.train.all-00000-of-00010_en_csim.tsv',
 'images',
 'wit_v1.train.all-00000-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00001-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00002-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00003-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00004-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00005-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00006-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00008-of-00010_context_caption_en_sbert_c.tsv']

In [28]:
fn = 'wit_v1.train.all-00000-of-00010_context_caption_en_sbert_c'

In [66]:
df = pd.read_csv(f'{root_path}/{fn}.tsv', sep='\t')
df = df[df["language"]=='en']
df = df.drop('Unnamed: 0', 1)
df = df.drop('index', 1)
df = df.reset_index(drop=True)
df

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,...,192_context_score,20_context_score,296_context_score,144_context_score,12_context_score,224_context_score,250_context_score,376_context_score,concept_caption_score,concept_context_score
0,en,https://en.wikipedia.org/wiki/Oxydactylus,https://upload.wikimedia.org/wikipedia/commons...,Oxydactylus,NaN,Oxydactylus,NaN,English: Mounted skeleton of Oxydactylus longi...,NaN,image/jpeg,...,-0.031757,0.069808,0.094782,-0.104758,0.121328,-0.131653,0.001038,0.111654,0.070058,0.016905
1,en,https://en.wikipedia.org/wiki/Maine%27s_3rd_co...,https://upload.wikimedia.org/wikipedia/commons...,Maine's 3rd congressional district,List of members representing the district,Maine's 3rd congressional district / List of m...,NaN,"English: Samuel W. Gould, US Representative fr...",NaN,image/jpeg,...,-0.003453,-0.007569,0.015267,0.044169,0.055935,-0.063109,0.046686,0.040697,0.133873,0.032929
2,en,https://en.wikipedia.org/wiki/Cheraw,https://upload.wikimedia.org/wikipedia/commons...,Cheraw,18th century,Cheraw / History / 18th century,A c. 1724 English copy of a deerskin Catawba m...,"English: ""Map of the Several Nations of Indian...",NaN,image/jpeg,...,0.011004,0.023219,0.151670,-0.067548,0.186415,0.049523,0.013646,0.038821,0.039021,-0.014457
3,en,https://en.wikipedia.org/wiki/Romanian_Front,https://upload.wikimedia.org/wikipedia/commons...,Romanian Front,Stagnation,Romanian Front / History / Stagnation,"Nameplate of Gazeta Transilvaniei on June 14, ...",English: Nameplate of the Romanian nationalist...,NaN,image/png,...,-0.022781,0.079207,-0.006843,0.033615,-0.082103,-0.054077,0.023136,-0.060647,0.046462,0.040290
4,en,https://en.wikipedia.org/wiki/%C3%81d%C3%A1m_K...,https://upload.wikimedia.org/wikipedia/commons...,Ádám Kósa,NaN,Ádám Kósa,NaN,English: Hungarian MEP Ádám Kósa,NaN,image/jpeg,...,0.122487,0.147636,0.043872,0.123579,0.046281,0.038244,0.195019,0.055083,0.207732,0.217288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542588,en,https://en.wikipedia.org/wiki/History_of_Latin,https://upload.wikimedia.org/wikipedia/commons...,History of Latin,Old Latin,History of Latin / Old Latin,"The Duenos inscription, from the 6th century B...",NaN,NaN,image/jpeg,...,-0.047678,-0.086387,-0.151684,-0.077020,-0.115906,-0.137534,-0.051402,-0.147671,-0.031749,-0.099965
542589,en,https://en.wikipedia.org/wiki/Battenberg_Castle,http://upload.wikimedia.org/wikipedia/commons/...,Battenberg Castle,NaN,Battenberg Castle,The Blitzröhren below the castle ruins,English: The so-called lightning tubes near Ba...,NaN,image/jpeg,...,-0.019007,-0.102026,0.050455,-0.028723,-0.072564,-0.017448,0.007824,0.056672,0.003922,-0.002685
542590,en,https://en.wikipedia.org/wiki/List_of_research...,https://upload.wikimedia.org/wikipedia/commons...,List of research vessels by country,National Oceanic and Atmospheric Administration,List of research vessels by country / United S...,"NOAAS Rainier with her namesake, Mount Rainier...",English: NOAAS Rainier (S 221) with Mount Rain...,NaN,image/jpeg,...,-0.033212,-0.046320,-0.000920,-0.052313,0.088083,0.032829,-0.029351,0.066505,0.180702,-0.032091
542591,en,https://en.wikipedia.org/wiki/Kino_Works,https://upload.wikimedia.org/wikipedia/commons...,Kino Works,NaN,Kino Works,NaN,English: Kino Works LLP Official Logo,NaN,image/jpeg,...,0.063145,0.025144,-0.017907,-0.005912,-0.071138,0.031493,0.077912,0.006186,0.037392,-0.063616


In [47]:
ped_cols = [el for el in list(df.columns) if '_score' in el]
org_cols = [el for el in list(df.columns) if not '_score' in el]
ped_cols = ['image_url'] + ped_cols
# print(cols)

In [37]:
corpus = df['caption'].tolist()
doc_len = len(corpus)
print(doc_len)

542593


In [49]:
def calc_all_sim(queries, corpus, doc_len):
    query_embeddings = embedder.encode(queries, convert_to_tensor=True)
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    
    corpus_embeddings = corpus_embeddings.to('cuda')
    corpus_embeddings = util.normalize_embeddings(corpus_embeddings)

    query_embeddings = query_embeddings.to('cuda')
    query_embeddings = util.normalize_embeddings(query_embeddings)
    
    hits = util.semantic_search(query_embeddings, corpus_embeddings, score_function=util.dot_score, top_k=doc_len)
    return hits

In [50]:
def extend_df(df, hits, q_id, test='caption'):
    for idx, hit in enumerate(hits):
        t1 = sorted(hit, key=lambda x: x['corpus_id'])
        df_ = pd.DataFrame(t1) 
        df_ = df_.drop(['corpus_id'], axis = 1)
        df_.columns = [f'{q_id[idx]}_{test}_score']
        df = pd.concat([df, df_], axis=1)
    return df

In [58]:
def extend_df_by_concept(df, hits, test='caption', concept=['car']):
    for idx, hit in enumerate(hits):
        t1 = sorted(hit, key=lambda x: x['corpus_id'])
        df_ = pd.DataFrame(t1) 
        df_ = df_.drop(['corpus_id'], axis = 1)
        df_.columns = [f'{concept[idx]}_{test}_score']
        df = pd.concat([df, df_], axis=1)
    return df

In [56]:
%%time
queries=['car', 'aircraft']
hits = calc_all_sim(queries, df['caption'].tolist(), len(df['caption'].tolist()))

CPU times: user 24min 7s, sys: 6min 52s, total: 31min
Wall time: 22min 42s


In [52]:
%%time
context_hits = calc_all_sim(queries, df['context'].tolist(), len(df['context'].tolist()))

CPU times: user 59min 11s, sys: 12min 4s, total: 1h 11min 15s
Wall time: 42min 5s


In [67]:
%%time
df = extend_df_by_concept(df, hits, test='caption', concept=queries)
df

CPU times: user 2.9 s, sys: 692 ms, total: 3.59 s
Wall time: 3.58 s


,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,...,296_context_score,144_context_score,12_context_score,224_context_score,250_context_score,376_context_score,concept_caption_score,concept_context_score,car_caption_score,aircraft_caption_score
0,en,https://en.wikipedia.org/wiki/Oxydactylus,https://upload.wikimedia.org/wikipedia/commons...,Oxydactylus,NaN,Oxydactylus,NaN,English: Mounted skeleton of Oxydactylus longi...,NaN,image/jpeg,...,0.094782,-0.104758,0.121328,-0.131653,0.001038,0.111654,0.070058,0.016905,0.042725,0.147009
1,en,https://en.wikipedia.org/wiki/Maine%27s_3rd_co...,https://upload.wikimedia.org/wikipedia/commons...,Maine's 3rd congressional district,List of members representing the district,Maine's 3rd congressional district / List of m...,NaN,"English: Samuel W. Gould, US Representative fr...",NaN,image/jpeg,...,0.015267,0.044169,0.055935,-0.063109,0.046686,0.040697,0.133873,0.032929,0.079170,0.101922
2,en,https://en.wikipedia.org/wiki/Cheraw,https://upload.wikimedia.org/wikipedia/commons...,Cheraw,18th century,Cheraw / History / 18th century,A c. 1724 English copy of a deerskin Catawba m...,"English: ""Map of the Several Nations of Indian...",NaN,image/jpeg,...,0.151670,-0.067548,0.186415,0.049523,0.013646,0.038821,0.039021,-0.014457,0.024344,-0.003153
3,en,https://en.wikipedia.org/wiki/Romanian_Front,https://upload.wikimedia.org/wikipedia/commons...,Romanian Front,Stagnation,Romanian Front / History / Stagnation,"Nameplate of Gazeta Transilvaniei on June 14, ...",English: Nameplate of the Romanian nationalist...,NaN,image/png,...,-0.006843,0.033615,-0.082103,-0.054077,0.023136,-0.060647,0.046462,0.040290,0.016165,0.053542
4,en,https://en.wikipedia.org/wiki/%C3%81d%C3%A1m_K...,https://upload.wikimedia.org/wikipedia/commons...,Ádám Kósa,NaN,Ádám Kósa,NaN,English: Hungarian MEP Ádám Kósa,NaN,image/jpeg,...,0.043872,0.123579,0.046281,0.038244,0.195019,0.055083,0.207732,0.217288,0.125367,0.165030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542588,en,https://en.wikipedia.org/wiki/History_of_Latin,https://upload.wikimedia.org/wikipedia/commons...,History of Latin,Old Latin,History of Latin / Old Latin,"The Duenos inscription, from the 6th century B...",NaN,NaN,image/jpeg,...,-0.151684,-0.077020,-0.115906,-0.137534,-0.051402,-0.147671,-0.031749,-0.099965,-0.017025,-0.028757
542589,en,https://en.wikipedia.org/wiki/Battenberg_Castle,http://upload.wikimedia.org/wikipedia/commons/...,Battenberg Castle,NaN,Battenberg Castle,The Blitzröhren below the castle ruins,English: The so-called lightning tubes near Ba...,NaN,image/jpeg,...,0.050455,-0.028723,-0.072564,-0.017448,0.007824,0.056672,0.003922,-0.002685,0.067823,0.061276
542590,en,https://en.wikipedia.org/wiki/List_of_research...,https://upload.wikimedia.org/wikipedia/commons...,List of research vessels by country,National Oceanic and Atmospheric Administration,List of research vessels by country / United S...,"NOAAS Rainier with her namesake, Mount Rainier...",English: NOAAS Rainier (S 221) with Mount Rain...,NaN,image/jpeg,...,-0.000920,-0.052313,0.088083,0.032829,-0.029351,0.066505,0.180702,-0.032091,0.158979,0.197102
542591,en,https://en.wikipedia.org/wiki/Kino_Works,https://upload.wikimedia.org/wikipedia/commons...,Kino Works,NaN,Kino Works,NaN,English: Kino Works LLP Official Logo,NaN,image/jpeg,...,-0.017907,-0.005912,-0.071138,0.031493,0.077912,0.006186,0.037392,-0.063616,0.051014,0.039353


In [68]:
#Sentences are encoded by calling model.encode()
emb1 = embedder.encode(df.at[0,'caption'])
emb2 = embedder.encode("aircraft")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.1470]])


In [ ]:
%%time
df = extend_df_by_concept(df, context_hits, test='context')

In [ ]:
df

In [70]:
%%time
f_idx = [0,1,2,3,4,5,6,7,8,9]
queries=['car', 'aircraft']
for fid in tqdm(f_idx):
    fn = f'wit_v1.train.all-0000{fid}-of-00010_context_caption_en_sbert_c'
    df = pd.read_csv(f'{root_path}/{fn}.tsv', sep='\t')
    df = df[df["language"]=='en']
    df = df.drop('Unnamed: 0', 1)
    df = df.reset_index(drop=True)

    corpus = df['caption'].tolist()
    doc_len = len(corpus)
    
    
    hits = calc_all_sim(queries, corpus, doc_len)
    
    
    
    df = extend_df_by_concept(df, hits, test='caption', concept=queries)
    
    df.to_csv(f'{root_path}/{fn}pa.tsv', sep="\t")
    os.listdir(root_path)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 4h 33min 51s, sys: 1h 12min 28s, total: 5h 46min 20s
Wall time: 4h 25min 46s


In [71]:
os.listdir(root_path)

['wit_v1.train.all-00007-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00009-of-00010_context_caption_en_sbert_c.tsv',
 '__MACOSX',
 'wit_v1.train.all-1percent_sample.tsv',
 'wit_v1.train.all-00000-of-00010_en_csim.tsv',
 'wit_v1.train.all-00006-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00008-of-00010_context_caption_en_sbert_cpa.tsv',
 'images',
 'wit_v1.train.all-00000-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00001-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00002-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00003-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00004-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00005-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00007-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00009-of-00010_context_caption_en_sbert_cpa.tsv',
 'wit_v1.train.all-00000-of-00010_context_caption_en_sbert_c.ts

In [ ]:
%%time
f_idx = [0,1,2,3,4,5,6,7,8,9]
for fid in f_idx:
    fn = f'wit_v1.train.all-0000{fid}-of-00010'
    df = pd.read_csv(f'{root_path}/{fn}.tsv', sep='\t')
    df = df[df["language"]=='en']
    df = df.reset_index()
    df['caption']=df['caption_reference_description'].fillna('') + ' ' + df['caption_attribution_description'].fillna('')
    df['context']=df['context_page_description'].fillna('') + '\n' + df['context_section_description'].fillna('')
    
    corpus = df['caption'].tolist()
    doc_len = len(corpus)
    
    hits = calc_all_sim(queries, corpus, doc_len)
    context_hits = calc_all_sim(queries, df['context'].tolist(), doc_len)
    
    df = extend_df(df, hits, q_id, test='caption')
    df = extend_df(df, context_hits, q_id, test='context')
    
    df.to_csv(f'{root_path}/{fn}_context_caption_en_sbert_.tsv', sep="\t")
    os.listdir(root_path)


CPU times: user 14h 34min 21s, sys: 2h 48min 30s, total: 17h 22min 52s
Wall time: 10h 41min 46s


In [174]:
%%time
f_idx = [0,1,2,3,4,5,6,7,8,9]
for fid in f_idx:
    fn = f'wit_v1.train.all-0000{fid}-of-00010'
    df = pd.read_csv(f'{root_path}/{fn}.tsv', sep='\t')
    df = df[df["language"]=='en']
    df = df.reset_index()
    df['caption']=df['caption_reference_description'].fillna('') + ' ' + df['caption_attribution_description'].fillna('')
    df['context']=df['context_page_description'].fillna('') + '\n' + df['context_section_description'].fillna('')
    corpus = df['caption'].tolist()
    doc_len = len(corpus)
    query_embeddings = embedder.encode(queries, convert_to_tensor=True)
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    
    corpus_embeddings = corpus_embeddings.to('cuda')
    corpus_embeddings = util.normalize_embeddings(corpus_embeddings)

    query_embeddings = query_embeddings.to('cuda')
    query_embeddings = util.normalize_embeddings(query_embeddings)
    hits = util.semantic_search(query_embeddings, corpus_embeddings, score_function=util.dot_score, top_k=doc_len)
    
    for idx, hit in enumerate(hits):
        t1 = sorted(hit, key=lambda x: x['corpus_id'])
        df_ = pd.DataFrame(t1) 
        df_ = df_.drop(['corpus_id'], axis = 1)
        df_.columns = [f'{q_id[idx]}_score']
        df = pd.concat([df, df_], axis=1)
    df.to_csv(f'{root_path}/{fn}_en_sbert.tsv', sep="\t")
    os.listdir(root_path)


CPU times: user 21min 52s, sys: 3min 51s, total: 25min 44s
Wall time: 15min 42s


In [44]:
df.describe()

,index,original_height,original_width,196_caption_score,412_caption_score,172_caption_score,363_caption_score,198_caption_score,114_caption_score,419_caption_score,...,192_context_score,20_context_score,296_context_score,144_context_score,12_context_score,224_context_score,250_context_score,376_context_score,concept_caption_score,concept_context_score
count,5.413720e+05,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,...,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000,541372.000000
mean,1.847620e+06,1526.483969,1823.540927,0.033204,0.028960,0.049363,0.009567,0.035998,0.046263,0.054036,...,0.003917,0.006184,0.039147,0.000788,0.010394,0.016874,0.019686,0.007192,0.087387,0.027541
std,1.069663e+06,1191.754269,1504.835260,0.068678,0.064804,0.068957,0.060760,0.074472,0.069364,0.076363,...,0.061797,0.067396,0.087407,0.060676,0.076750,0.088751,0.069646,0.077725,0.070005,0.066885
min,1.500000e+01,100.000000,100.000000,-0.241611,-0.230843,-0.216518,-0.241853,-0.248987,-0.260702,-0.241586,...,-0.254211,-0.266523,-0.272346,-0.247377,-0.271537,-0.328301,-0.255366,-0.380909,-0.223028,-0.268890
25%,9.209935e+05,576.000000,640.000000,-0.013408,-0.015514,0.003183,-0.031997,-0.015480,-0.001160,0.002301,...,-0.038302,-0.039394,-0.021247,-0.041057,-0.041979,-0.044542,-0.028180,-0.045769,0.039938,-0.017910
50%,1.846164e+06,1169.000000,1300.000000,0.027043,0.025379,0.044976,0.007512,0.031951,0.043219,0.047380,...,0.002984,0.004428,0.032268,-0.001115,0.006349,0.010801,0.018052,0.005007,0.086895,0.026056
75%,2.771949e+06,2285.000000,2643.000000,0.072334,0.068956,0.089922,0.048739,0.082723,0.090399,0.097352,...,0.044986,0.049166,0.091044,0.040647,0.058230,0.072214,0.066000,0.057772,0.134229,0.070613
max,3.704241e+06,21851.000000,31281.000000,0.755007,0.506310,0.553052,0.412655,0.532263,0.661468,0.554242,...,0.475118,0.478698,0.560603,0.332110,0.485849,0.549571,0.360359,0.528677,0.570127,0.548307


In [172]:
os.listdir(root_path)

['wit_v1.train.all-1percent_sample.tsv',
 'wit_v1.train.all-00000-of-00010.tsv',
 'wit_v1.train.all-00000-of-00010_en_csim.tsv',
 'wit_v1.train.all-00001-of-00010.tsv',
 'wit_v1.train.all-00000-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00001-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00002-of-00010.tsv',
 'wit_v1.train.all-00002-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00003-of-00010.tsv',
 'wit_v1.train.all-00003-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00004-of-00010.tsv',
 'wit_v1.train.all-00005-of-00010.tsv',
 'wit_v1.train.all-00006-of-00010.tsv',
 'wit_v1.train.all-00007-of-00010.tsv',
 'wit_v1.train.all-00008-of-00010.tsv',
 'wit_v1.train.all-00009-of-00010.tsv',
 'wit_v1.train.all-00004-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00005-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00006-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00007-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00008-of-00010_en_sbert.tsv',
 'wit_v1.train.all-00009-of-00010_en_sbert.tsv']

In [103]:
df.at[54070,'caption']

'S. quiescens forms mycorrhizal associations with Bishop Pine. English: Pinus muricata at the Dundee Botanic Garden'

In [101]:
queries[-1]

'pedestrian peanut bear mickey charlie snoopy disney cartoon trolley lucy pedestrian'

In [104]:
print("Cosine-Similarity:", get_sim(queries[-1], embedder.encode(df.at[54070,'caption'])))

Cosine-Similarity: 0.018515588715672493
